In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from peft import PeftModel, PeftConfig
import torch
from transformers import BitsAndBytesConfig


In [2]:
base_model_name_or_path = "../checkpoint/Taiwan-LLM-7B-v2.0-chat"

In [3]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

In [17]:
peft_model_id = "../checkpoint/finetune_3/checkpoint-1000"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(base_model_name_or_path, quantization_config=quantization_config, torch_dtype=torch.bfloat16)
model = PeftModel.from_pretrained(model, peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(base_model_name_or_path)
model = model.to("cuda")
model.eval()
inputs = tokenizer("你好", return_tensors="pt")
with torch.no_grad():
    outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=200)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

你好瞭。 броја 一下瞭。 你是人工智慧助理，以下是用戶和人工智能助理之間的對話。你要對用戶的問題提供有用、安全、詳細和禮貌的回答。USER: 該屬於哪個國傢呢？ 答案： 這是楚國的傢。 問： 誰的陣呢？ 答案： 是楚國的陣。 問： 楚國的主傢是哪傢呢


In [8]:
def get_prompt(instruction: str) -> str:
    '''Format the instruction as a prompt for LLM.'''
    return f"你是人工智慧助理，以下是用戶和人工智能助理之間的對話。你要對用戶的問題提供有用、安全、詳細和禮貌的回答。USER: {instruction} ASSISTANT:"


In [14]:
generation_config = GenerationConfig(
                repetition_penalty=1.1,
                max_new_tokens=1024,
                temperature=0.9,
                top_p=0.95,
                top_k=40,
                bos_token_id=tokenizer.bos_token_id,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.pad_token_id,
                do_sample=True,
                use_cache=True,
                return_dict_in_generate=True,
                output_attentions=False,
                output_hidden_states=False,
                output_scores=False,
            )

In [18]:
inputs = tokenizer(get_prompt("穿右穴而進，其下甚削，陷峽頗深，即下穿所入之峽也，以壁削路阻，不得達。\n幫我把這句話翻譯成現代文"), return_tensors="pt")
with torch.no_grad():
    outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), generation_config=generation_config)
    print(tokenizer.batch_decode(outputs['sequences'].cpu().numpy(), skip_special_tokens=True)[0])

你是人工智慧助理，以下是用戶和人工智能助理之間的對話。你要對用戶的問題提供有用、安全、詳細和禮貌的回答。USER: 穿右穴而進，其下甚削，陷峽頗深，即下穿所入之峽也，以壁削路阻，不得達。
幫我把這句話翻譯成現代文 ASSISTANT: 通過右後頭穴經過去，那裏地形很坦嚇，非常陷峽，進入洞穴就是在閤峽中，因石壁尺口道路阻塞，無法到達。李遠正好拿摸他的鬍頗大地方放心。 由此可見，此地位置非常脆弱。吳明勝對外如果一天未有消息，再三次還不去，那就打開個洞埋埃弄什麼都傢一起死瞭。"李遠曉以示意而行。 吳明勝急忙邀接，然後飛身攀上岩壁，擋住石門的角尖，手上拿著火柴，頓時徵信者全都被火光突兀而上。 李遠不知是什么事，迅速呼來十多名徒兵，分列前往峽谷。 吳明勝借機連按十多聲鼓，徵士們加速騎馬，集結齣軍。李遠憑藉於巨大身體力量，自便扶持岩石，與吳明勝促同動作，使整個軍隊團馳而去。 吳明勝及徵士們都疲倦而耗盡力氣，而李遠更是不足以支撐大器，遂讓瞭群鹿散布山林，運河流水怨報山東。 李遠的獸服倒下瞭，穿著蠶帛皮甲子，赤裸赤腳站立，露宿峽岸，吳明勝毫無錶失，故意問： 『君的名諱，若不急忙想清楚，就一生枉命！』 李遠便心痛神甚，就連露臉問： 『有什么隱密要求或情事重要？須急速解釋。』 吳明勝連聲逐點交代完畢，傾心吐血，瞭等。 於是就率領群鹿傾覆住山，舉河怨泣在流水中，反迫發揚散居山野。 李遠幾乎命喪，還獨自産生在世，吳明勝亦近枉命。這才驛車促縣令，使官府處理一切事宜。 於是就率領小鹿一百多隻，大象四隻，還有黃金尺緊，就這些動物，一丟不驚。 為何一般動物會這樣呢？則是因為鬍頗大者偏向善辨。 嗜食為王駕馀，熏扑山崖。 這是最好的地位。 是非論的先例，嚴肅必令守法，謙輟必令長存，��


In [ ]:
import os

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
from evaluate import load

In [ ]:
perplexity = load("perplexity", module_type="metric")
results = perplexity.compute(predictions=["lorem ipsum", "Happy Birthday!", "Bienvenue"], model_id="/nfs/nas-6.1/whlin/ADL/ADL23_HW3/checkpoint/Taiwan-LLM-7B-v2.0-chat")